# **3η ομαδική εργασία στη Τεχνολογία και Ανάλυση Εικόνων και Βίντεο**

## **Στοιχεία μελών ομάδας**

Βασίλειος Βαρσαμής - 03118033

Αλέξανδρος Κουλάκος - 03118144

## **Θεωρητικό μέρος**

### Ερώτηση 1

**Q**: Ποια είναι η διαφορά του bounding box και του anchor box στο YOLO;

**A**: To bounding box καθορίζεται από τη διαδικασία εκπαίδευσης και επιλέγεται ως το καλύτερο για κάθε κλάση χρησιμοποιώντας τον non-max suppression αλγόριθμο. **Χρησιμοποιώντας μόνο bounding boxes κάθε grid cell μπορεί να πρόβλεψει το πολύ μία κλάση**, ωστόσο σε πολλές εικόνες υπάρχει επικάλυψη των εικονιζόμενων επιμέρους οντοτήτων (δηλαδή τα κέντρα τους ανήκουν στο ίδιο κελί) και έτσι η μέθοδος τα ταξινομεί σε μία κλάση, αγνοώντας τις υπόλοιπες. Η χρήση των anchor boxes έρχεται να λύσει το παραπάνω πρόβλημα με τον εξής τρόπο: ορίζουμε πριν την εκπαίδευση ένα σύνολο από bounding boxes δεδομένου ύψους και πλάτους, τα οποία θέλουμε να προσομοιάζουν το σχήμα των κλάσεων που προσδιορίζουν (anchor boxes). Το Νευρωνικό Δίκτυο προβλέπει κάποιo offset από τις διαστάσεις του anchor box για τις διαστάσεις του υποψήφιου κουτιού και με βάση μια συνάρτηση απώλειας κάνει σταδιακά καλύτερη αυτήν την πρόβλεψη. Με αυτόν τον τρόπο καθίσταται δυνατή η αναγνώριση πολλών αντικειμένων, που έχουν διαφορετική κλίμακα ή/και που επικαλύπτονται.

### Ερώτηση 2

**Q**: Ποιες θα είναι οι διαστάσεις του πίνακα εξόδου (πρόβλεψης) y_hat του αλγορίθμου YOLO
θεωρώντας ότι έχουμε δυο anchor boxes και 3 κλάσεις; Αναφέρετε επίσης το ρόλο για κάθε
στοιχείο του πίνακα αυτού.

**A**: Αν θέσουμε $Β$ το πλήθος των anchor boxes, $Ν$ τη διάσταση του grid και $C$ το πλήθος των κλάσεων προς ταξινόμηση, ο τένσορας της εξόδου του αλγορίθμου YOLO έχει διαστάσεις $ N × N × B\cdot(C + 5) $, διότι για κάθε grid cell θα έχουμε ένα διάνυσμα $y$ που θα αποτελείται από $ B\cdot(C + 5) $ θέσεις. Συγκεκριμένα, οι θέσεις του διανύσματος $y$ έχουν ως εξής:
 
•	1 για το detection probability $p_c$ (δηλαδή την πιθανότητα να βρούμε μια συγκεκριμένη κλάση μέσα στο εν λόγω grid cell)

•	4 για τις συντεταγμένες του bounding box

•	$C$ για τις προς αναγνώριση κλάσεις

Και τα παραπάνω $B$ φορές, μία για κάθε anchor box.

Επομένως τελικά, αν περιοριστούμε σε ένα grid cell για $ B = 2 $ και $ C = 3 $ έχουμε στην έξοδο διάνυσμα με διαστάσεις $ 1 × 16 $.


### Ερώτηση 3

**Q**: Εξηγήστε συνοπτικά την έννοια της μεθόδου Non-max suppression. Σε τι χρησιμεύει;

**A**: Η non-max suppression είναι μια τεχνική που χρησιμοποιείται σε προβλήματα υπολογιστικής όρασης και αποτελείται από μία κλάση αλγορίθμων που επιλέγουν μία οντότητα (εδώ bounding boxes) από πολλές υποψήφιες. Η επιλογή αυτή γίνεται με κάποια κριτήρια που στην περίπτωση μας είναι δύο: α) η πιθανότητα ανίχνευσης, β) κάποια μετρική επικάλυψης (εδώ η IoU).

Η ιδέα της τεχνικής αυτής που περιγράφεται παρακάτω είναι εξαιρετικά απλή. Ο αλγόριθμος εξελίσσεται σε δύο βήματα:

1)	Από την λίστα των προβλέψεων (έστω $P$) για τα bounding boxes παίρνουμε αυτή που έχει την μεγαλύτερη τιμή $p_c$ και την τοποθετούμε στην τελική λίστα (έστω $Κ$).

2)	Για όλες τις εναπομείνουσες προβλέψεις στην $P$ υπολογίζουμε την IoU με αυτήν που βάλαμε στη λίστα $Κ$ και απορρίπτουμε όλες αυτές που έχουν IoU μεγαλύτερο από κάποιο threshold (π.χ 0.5) και συνεχίζουμε από το βήμα 1 μέχρι να αδειάσει η λίστα $P$.
 
Διαισθητικά ο παραπάνω αλγόριθμος κάνει το παρακάτω (για κάθε κλάση):

Κρατάει την καλύτερη πρόβλεψη (αυτήν με το μεγαλύτερο $p_c$), δηλαδή την maximal, και κάθε λύση που δεν είναι maximal, αλλά επικαλύπτεται ποσοστιαία παραπάνω από κάποιο ορισμένο κατώφλι με αυτήν που κρατήσαμε, την απορρίπτουμε (δηλαδή suppress the non - maximal). Πράγματι, είναι λογικό να την απορρίπτουμε, γιατί έχουμε ήδη μία πρόβλεψη (αυτή που κρατήσαμε) με καλύτερη τιμή του $p_c$ και που μοιάζει πολύ χωρικά με αυτή που απορρίπτουμε, εφόσον έχει IoU μεγαλύτερο από κάποιο κατώφλι. Άρα, και στις δύο παραμέτρους η πρόβλεψη που απορρίψαμε “χάνει” απέναντι σε αυτήν που κρατήσαμε.


### Ερώτηση 4

**Q**: Στον αλγόριθμο SORT το ταίριασμα (matching) των αντικειμένων με ποια μέθοδο γίνεται;
Αναφέρετε ένα παράδειγμα μετρικής για το bounding boxes distance.


**A**: Στον αλγόριθμο SORT το ταίριασμα (matching) των αντικειμένων γίνεται με τη μέθοδο του Hungarian algorithm. Ένα παράδειγμα μετρικής για το bounding boxes distance αποτελεί η ΙοU (Intersection over Union) που στην ουσία ταυτίζεται με την ομοιότητα Jaccard για δύο σύνολα $ A, B $ που ορίζεται ως: $J = \frac{|Α \cap Β|}{|A \cup B|}$, όπου εδώ τα $ Α, B $ είναι τα σύνολα των pixels που καταλαμβάνουν τα δύο bounding boxes.

### Ερώτηση 5

**Q**: Πώς ορίζεται η μέθοδος του tracking-by-detection και σε ποιο component αυτής της
μεθόδου χρησιμοποιείται βαθιά μάθηση (deep learning);

**A**: Η μέθοδος του tracking by detection ορίζεται ως εξής: έχουμε κάποια καρέ εικόνας ενός βίντεο. Πάνω σε αυτές, εφαρμόζουμε έναν ανιχνευτή ο οποίος δίνει πιθανά bounding boxes που περιέχουν στο εσωτερικό τους κάποια κλάση (στην εργασία που δίνεται πρόκειται για πεζούς). Αυτό γίνεται για κάθε καρέ και χρησιμοποιώντας CNN αρχιτεκτονικές (όπως το YOLO ή άλλα FrCNNs). Έπειτα, με την χρήση φίλτρων Kalman καθώς και του Hungarian αλγορίθμου, γίνεται το tracking, δηλαδή το data association (σύνδεση των ανεξάρτητων ανιχνεύσεων κάθε καρέ) για να πάρουμε την πλήρη τροχιά. To component της παραπάνω μεθόδου, όπως προαναφέραμε, στο οποίο χρησιμοποιείται deep learning είναι το detection.

## **Εργαστηριακό μέρος**

### **Προετοιμασία των απαραίτητων πακέτων/βιβλιοθηκών**

In [ ]:
import sys

!{sys.executable} -m pip install torch==1.5.0 torchvision==0.6.0
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install filterpy==1.4.5
!{sys.executable} -m pip install scikit-image==0.17.2
!{sys.executable} -m pip install lap==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Mount Google Drive filesystem
# to access necessary python modules

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 

# Absolute path to working directory
path = '/content/drive/My Drive/Image_Video_Technology/Tasks/Task_3/Lab_motion_tracking_exercise/'

# Interpreter will search in the working directory for the necessary modules
sys.path.insert(0, path)

# Check that all necessary files/folders exist
print(os.listdir(path))

['models.py', 'sort.py', 'group_92.mp4', 'config', 'images', 'utils', '__pycache__', 'Object_Detection_and_Tracking.ipynb', 'img_Lab3.ipynb']


In [ ]:
from models import *
from utils import *
from sort import *

import time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image

import cv2

In [ ]:
print("Current working directory:", os.getcwd())

Current working directory: /content


### Κατασκευή των βοηθητικών συναρτήσεων

In [ ]:
# Function that uses a pre-trained Neural Network to make image detections 
# for specified values of conf_thres and nms_thres.

def detect_image(img, conf_thres, nms_thres):
    # scale and pad image
    ratio = min(img_size / img.size[0], img_size / img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)

    img_transforms = transforms.Compose([ 
                                         transforms.Resize((imh, imw)),
                                         transforms.Pad((max(int((imh - imw) / 2), 0), max(int((imw - imh) / 2), 0), 
                                                         max(int((imh - imw) / 2), 0), max(int((imw - imh) / 2), 0)), 
                                                        (128, 128, 128)),
                                         transforms.ToTensor(),
                                          ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    
    return detections[0]

In [ ]:
# Function that uses a pre-trained Neural Network to make and plot detections in a video (frame by frame) 
# for specified values of conf_thres and nms_thres.

def object_tracking(conf_thres, nms_thres):
  vid = cv2.VideoCapture(videopath)
  mot_tracker = Sort() 
  frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)

  for ii in range(int(frames)):
      ret, frame = vid.read()
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      pilimg = Image.fromarray(frame)
      detections = detect_image(pilimg, conf_thres, nms_thres)

      img = np.array(pilimg)
      pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
      pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
      unpad_h = img_size - pad_y
      unpad_w = img_size - pad_x

      if (detections is not None) and (ii % 20 == 0):
          print("ii", ii)
          print(detections.shape)
          tracked_objects = mot_tracker.update(detections.cpu())
          print(tracked_objects.shape)

          unique_labels = detections[:, -1].cpu().unique()
          n_cls_preds = len(unique_labels)
          for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
              box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
              box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
              y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
              x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

              color = colors[int(obj_id) % len(colors)]
              color = [i * 255 for i in color]
              cls = classes[int(cls_pred)]
              cv2.rectangle(frame, (x1, y1), (x1 + box_w, y1 + box_h), color, 4)
              cv2.rectangle(frame, (x1, y1 - 35), (x1 + len(cls) * 19 + 60, y1), color, -1)
              cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
      
      if ii % 20 == 0:
        fig=plt.figure(figsize=(12, 8))
        plt.title("Video Stream")
        plt.imshow(frame)
        plt.show()

### Πειραματισμός με τις υπερπαραμέτρους `conf_thres` και `nms_thres`

In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
# Load parameters of the Neural Network
config_path = path + 'config/yolov3.cfg'

# Load weights of the pre-trained model YOLOv3 on COCO dataset
weights_path = path + 'config/yolov3.weights'

# Load classes for detection (80 classes in total)
class_path = path + 'config/coco.names'

img_size = 416

# Load DarkNet model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

# Absolute path to group video
videopath = path + 'group_92.mp4'

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Λόγω του πολύ μεγάλου output (το βίντεο συντίθεται από 300 καρέ), επιλέγουμε να εμφανίζουμε τα καρέ ανά 20 (δηλαδή, σε κάθε πειραματισμό τυπώνονται $ 300/20 = 15 $ εικόνες). Πράγματι, αυτή η δειγματοληψία είναι πρακτική, διότι τα εμφανιζόμενα καρέ είναι τελείως αντιπροσωπευτικά όσον αφορά τα συμπεράσματα που αναλύονται παρακάτω.

#### conf_thres = 0.3, nms_thres = 0.4

In [ ]:
object_tracking(0.3, 0.4)

Output hidden; open in https://colab.research.google.com to view.

#### conf_thres = 0.9, nms_thres = 0.4

In [ ]:
object_tracking(0.9, 0.4)

Output hidden; open in https://colab.research.google.com to view.

#### conf_thres = 0.8, nms_thres = 0.3

In [ ]:
object_tracking(0.8, 0.3)

Output hidden; open in https://colab.research.google.com to view.

#### conf_thres = 0.8, nms_thres = 0.9

In [ ]:
object_tracking(0.8, 0.9)

Output hidden; open in https://colab.research.google.com to view.

#### Συμπεράσματα

##### Confidence threshold

Bounding boxes για τα οποία ισχύει ότι `conf_score < conf_thres` (δηλαδή η πιθανότητα ανίχνευσης αντικειμένου είναι μικρότερη από την τιμή κατωφλίου που ορίζουμε εμείς) δε λαμβάνονται υπόψη στο πρόβλημα του image detection. Έτσι, φαίνεται ότι το μέγεθος `conf_thres` διαδραματίζει καθοριστικό ρόλο όσον αφορά το σύνολο των bounding boxes που θεωρούμε ότι ανιχνεύουν ικανοποιητικά κάποιο αντικείμενο, δηλαδή διαφορετικές τιμές του `conf_thres` καθορίζουν την ποιότητα και την ορθότητα της ανίχνευσης. Πράγματι, αυτή η διαπίστωση δικαιολογείται από τα αποτελέσματα της προσομοίωσης. Συγκεκριμένα, παρατηρούμε ότι για μικρές τιμές του `conf_thres` (π.χ. `conf_thres = 0.3`), σε κάθε καρέ του βίντεο επισημαίνονται πάρα πολλά bounding boxes (σε βαθμό που τα καθιστά έως και δυσδιάκριτα μεταξύ τους) και επίσης πολλά bounding boxes κάνουν λανθασμένες προβλέψεις (βλ. καρέ υπ' αριθμόν 20, 80 και 100), που σημαίνει ότι λόγω της χαμηλής τιμής `conf_thres` που ορίσαμε επιτρέπουμε σε πολλά bounding boxes να ανιχνεύουν αντικείμενα, ωστόσο όχι και τόσο ικανοποιητικά. Αντίθετα, παρατηρούμε ότι για μεγάλες τιμές του `conf_thres` (π.χ. `conf_thres = 0.9`), σε κάθε καρέ του βίντεο επισημαίνονται πλέον πολύ λίγα bounding boxes τα οποία κάνουν ολόσωστες προβλέψεις, που σημαίνει ότι λόγω της υψηλής τιμής `conf_thres` που ορίσαμε επιτρέπουμε σε λίγα bounding boxes να ανιχνεύουν αντικείμενα και μάλιστα με ιδιαίτερα ικανοποιητικό τρόπο. 

**Συνεπώς, μέσα από διάφορους πειραματισμούς, είναι καίριο να προσδιορίσουμε εκείνη την τιμή του `conf_thres` η οποία οδηγεί στην ύπαρξη πολλών bounding boxes (δηλαδή στην ανίχνευση πολλών αντικειμένων), αλλά ταυτόχρονα με καλή ακρίβεια. Συνήθως, μια καλή τέτοια τιμή είναι `conf_thres = 0.7` ή `conf_thres= 0.8`.**

##### Non-max Suppression threshold

Έστω ότι βρισκόμαστε στο στάδιο non-max suppression (κατά το οποίο σκοπός είναι να απαλείψουμε bounding boxes που αναφέρονται στο ίδιο αντικείμενο) και έχουμε επιλέξει το bounding box με το υψηλότερο `conf_score`. Τότε, βρίσκουμε το `IoU` αυτού του box με όλα τα υπόλοιπα bounding boxes και απορρίπτουμε εκείνα τα boxes για τα οποία ισχύει `IoU > nms_thres`. Συνεπώς, προκύπτει ότι όσο μικρότερο είναι η τιμή `nms_thres` που θέτουμε (δηλαδή όσο πιο "αυστηροί" είμαστε), τόσο πιο λίγα bounding boxes που προσδιορίζουν το ίδιο αντικείμενο γίνονται αποδεκτά, ενώ όσο μεγαλύτερη είναι η τιμή `nms_thres` που θέτουμε (δηλαδή όσο πιο "ελαστικοί" είμαστε), τόσο πιο πολλά bounding boxes που προσδιορίζουν το ίδιο αντικείμενο γίνονται αποδεκτά. Πράγματι, αυτή η διαπίστωση δικαιολογείται από τα αποτελέσματα της προσομοίωσης. Συγκεκριμένα, παρατηρούμε ότι για μικρές τιμές του `nms_thres` (π.χ. `nms_thres = 0.3`), στα περισσότερα καρέ του βίντεο κάθε αντικείμενο πλαισιώνεται (ανιχνεύεται) από ένα και μόνο bounding box, ενώ για μεγάλες τιμές του `nms_thres` (π.χ. `nms_thres = 0.9`) στα περισσότερα καρέ υπάρχουν αντικείμενα (π.χ. bus) που πλαισιώνονται από παραπάνω από ένα bounding boxes.

**Συνεπώς, προκύπτει ότι είναι βολικό και αποδοτικό να επιλέγουμε μικρές τιμές για το nms_thres, έτσι ώστε να είναι αποτελεσματική η απαλοιφή bounding boxes που ανιχνεύουν το ίδιο αντικείμενο. Συνήθως, καλές τέτοιες τιμές είναι `nms_thres = 0.3` και `nms_thres = 0.4`.**